In [ ]:
import numpy as np
import torch
import math
import pandas as pd

In [ ]:
data = pd.read_csv("FuelConsumptionCo2.csv")
data.head

<bound method NDFrame.head of       MODELYEAR   MAKE       MODEL    VEHICLECLASS  ENGINESIZE  CYLINDERS  \
0          2014  ACURA         ILX         COMPACT         2.0          4   
1          2014  ACURA         ILX         COMPACT         2.4          4   
2          2014  ACURA  ILX HYBRID         COMPACT         1.5          4   
3          2014  ACURA     MDX 4WD     SUV - SMALL         3.5          6   
4          2014  ACURA     RDX AWD     SUV - SMALL         3.5          6   
...         ...    ...         ...             ...         ...        ...   
1062       2014  VOLVO    XC60 AWD     SUV - SMALL         3.0          6   
1063       2014  VOLVO    XC60 AWD     SUV - SMALL         3.2          6   
1064       2014  VOLVO    XC70 AWD     SUV - SMALL         3.0          6   
1065       2014  VOLVO    XC70 AWD     SUV - SMALL         3.2          6   
1066       2014  VOLVO    XC90 AWD  SUV - STANDARD         3.2          6   

     TRANSMISSION FUELTYPE  FUELCONSUMPTION_CITY  FUELCONSUMPTION_HWY  \
0             AS5        Z                   9.9                  6.7   
1              M6        Z                  11.2                  7.7   
2             AV7        Z                   6.0                  5.8   
3             AS6        Z                  12.7                  9.1   
4             AS6        Z                  12.1                  8.7   
...           ...      ...                   ...                  ...   
1062          AS6        X                  13.4                  9.8   
1063          AS6        X                  13.2                  9.5   
1064          AS6        X                  13.4                  9.8   
1065          AS6        X                  12.9                  9.3   
1066          AS6        X                  14.9                 10.2   

      FUELCONSUMPTION_COMB  FUELCONSUMPTION_COMB_MPG  CO2EMISSIONS  
0                      8.5                        33           196  
1                      9.6                        29           221  
2                      5.9                        48           136  
3                     11.1                        25           255  
4                     10.6                        27           244  
...                    ...                       ...           ...  
1062                  11.8                        24           271  
1063                  11.5                        25           264  
1064                  11.8                        24           271  
1065                  11.3                        25           260  
1066                  12.8                        22           294  

[1067 rows x 13 columns]>

In [ ]:
X = torch.tensor(data["ENGINESIZE"].values, dtype=torch.float32)
X2 = torch.tensor(data["FUELCONSUMPTION_COMB"].values, dtype=torch.float32)
Y = torch.tensor(data["CO2EMISSIONS"].values, dtype=torch.float32)
X = (X - X.mean()) / X.std()
X2 = (X2 - X2.mean()) / X2.std()
Y = (Y - Y.mean()) / Y.std()
inputs = torch.stack([X, X2], dim=1)
class MyModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1=torch.nn.Linear(2,3)
    self.fc2=torch.nn.Linear(3,4)
    self.fc3=torch.nn.Linear(4,1)
    self.flatten=torch.nn.Flatten(0,1)
  def forward(self,input):
    first=self.fc1(input)
    middle=self.fc2(first)
    out=self.fc3(middle)
    out=self.flatten(out)
    return out
model=MyModel()
loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate=1e-6
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
for t in range(1000):
    y_pred = model(inputs)
    loss = loss_fn(y_pred, Y)
    if t % 100 == 99:
      print(t, loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(list(model.parameters()))

99 635.668701171875
199 461.35650634765625
299 346.4585876464844
399 275.10638427734375
499 232.61468505859375
599 207.90016174316406
699 193.65476989746094
799 185.3741455078125
899 180.40882873535156
999 177.25885009765625
[Parameter containing:
tensor([[ 0.7004,  0.4301],
        [ 0.6854,  0.4688],
        [-0.6088,  0.3837]], requires_grad=True), Parameter containing:
tensor([ 0.5853,  0.3674, -0.6056], requires_grad=True), Parameter containing:
tensor([[ 0.2553, -0.0152,  0.1059],
        [ 0.5339,  0.5771, -0.1362],
        [ 0.1914, -0.3215,  0.3777],
        [-0.1053, -0.5326,  0.5207]], requires_grad=True), Parameter containing:
tensor([ 0.0732,  0.0615, -0.1936,  0.1868], requires_grad=True), Parameter containing:
tensor([[ 0.2037,  0.5011, -0.1230, -0.2314]], requires_grad=True), Parameter containing:
tensor([-0.4819], requires_grad=True)]
